In [ ]:
import pandas as pd
import re

**Data Cleaning and Analysis Steps**:

* Calculated `total` for each transaction as `quantity * price`.
* Aggregated `total_spent` per customer using `groupby`.
* Counted the number of purchases (`purchase_frequency`) per customer.
* Ranked transactions by date per customer to determine the most recent purchase.
* Calculated `recency` as the time difference between the most recent purchase and a reference date.
* Merged all metrics (`total_spent`, `purchase_frequency`, `recency`) into a final customer summary dataset.


In [ ]:
df = pd.read_csv("online_retail_listing2.csv", sep=';', decimal=',')
df.head(25)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,1.12.2009 07:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,1.12.2009 07:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,1.12.2009 07:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,1.12.2009 07:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.12.2009 07:45,1.25,13085.0,United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,1.12.2009 07:45,1.65,13085.0,United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,1.12.2009 07:45,1.25,13085.0,United Kingdom
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,1.12.2009 07:45,5.95,13085.0,United Kingdom
8,489435,22350,CAT BOWL,12,1.12.2009 07:46,2.55,13085.0,United Kingdom
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,1.12.2009 07:46,3.75,13085.0,United Kingdom


In [ ]:
df.columns = [x.lower().replace(" ", "_").replace("?", "")\
.replace(r"/", "").replace("\\", "").replace("-", "_")\
.replace("%", "").replace(r",", "") for x in df.columns]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   invoice      1009 non-null   object 
 1   stockcode    1009 non-null   object 
 2   description  1008 non-null   object 
 3   quantity     1009 non-null   int64  
 4   invoicedate  1009 non-null   object 
 5   price        1009 non-null   float64
 6   customer_id  1003 non-null   float64
 7   country      1009 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 63.2+ KB


In [ ]:
missing_data = df.isnull().sum()
print(missing_data)

invoice        0
stockcode      0
description    1
quantity       0
invoicedate    0
price          0
customer_id    6
country        0
dtype: int64


In [ ]:
df = df.dropna()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1003 entries, 0 to 1008
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   invoice      1003 non-null   object 
 1   stockcode    1003 non-null   object 
 2   description  1003 non-null   object 
 3   quantity     1003 non-null   int64  
 4   invoicedate  1003 non-null   object 
 5   price        1003 non-null   float64
 6   customer_id  1003 non-null   float64
 7   country      1003 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 70.5+ KB


In [ ]:
df['date'] = pd.to_datetime(df['invoicedate'], format='%d.%m.%Y %H:%M')

Moving on to the main task — FRM analysis

I create a new row with the total amount, which I calculate by multiplying the quantity of goods by their price.

FRM analysis allows to estimate income and expenses, which helps in further decision-making.

In [ ]:
df['total'] = df['quantity']*df['price']

I group the date by the buyer's identification number and add up the total value per buyer + I reset the existing indexation to introduce a new one (after previous corrections).

In [ ]:
m = df.groupby('customer_id')['total'].sum()
m = pd.DataFrame(m).reset_index()

Rename the column for total cost and select the identification numbers and their total costs from frame m.

In [ ]:
m.rename(columns={'total': 'total_spent'}, inplace=True)
result = m[['customer_id', 'total_spent']]

Grouping dates by invoices for each buyer to assess customer activity.
I am resetting the indexing again to make it easier to work with.

In [ ]:
freq = df.groupby('customer_id')['invoicedate'].count()
f = pd.DataFrame(freq).reset_index()

In [ ]:
f.rename(columns={'invoicedate': 'purchase_frequency'}, inplace=True)

Sorting and grouping customers + assigning a rank to purchases using the min method so that the same rank is assigned to the same dates (purchases made at the same time).


In [ ]:
df['rank'] = df.sort_values(['customer_id', 'date']).groupby(['customer_id'])['date'].rank(method='min', ascending=True)

Extraction of liquids where the previous function returned 1 - the very first buyers with recent purchases

In [ ]:
recent = df[df['rank'] == 1].copy()

Calculates the difference between the date of the first purchase and the specified date + saves the result in the last created frame

In [ ]:
recent['recency'] = recent['date']-pd.to_datetime('2009-01-12 07:45:00')

In [ ]:
recent = recent[['customer_id', 'recency']].drop_duplicates()

Combining previously created data frames f / m / recent by the buyer's identification number column + output + loading the final version into a new final file

In [ ]:
online_retail_listing_final = f.merge(m,on='customer_id').merge(recent,on='customer_id')
print("\n")
print(online_retail_listing_final)



    customer_id  purchase_frequency  total_spent           recency
0       12362.0                   1       130.00 323 days 02:25:00
1       12533.0                  44       929.92 323 days 04:05:00
2       12615.0                   1        -5.95 323 days 04:24:00
3       12636.0                   1       141.00 323 days 02:10:00
4       12682.0                  19       426.30 323 days 01:43:00
5       12913.0                   5       537.96 323 days 04:33:00
6       12931.0                   1      1160.00 323 days 04:01:00
7       13078.0                  19       630.33 323 days 01:21:00
8       13085.0                  12       651.10 323 days 00:00:00
9       13293.0                   1       -12.60 323 days 03:10:00
10      13394.0                  20       304.53 323 days 04:13:00
11      13635.0                  23       382.37 323 days 02:01:00
12      13758.0                  27       996.10 323 days 02:21:00
13      13767.0                  19      1197.80 323 days 03

As a result, we have 50 of the most important customers of this store, who are regular buyers with purchases ranging from 12000 to 18000 with total amount of spends from -4 (what can be a refund) to 2286.

Identifying the top 50 customers who are regular buyers with high total purchases allows the business to:
- Tailor promotions, loyalty programs, and personalized offers to retain these high-value customers.
- Increase customer lifetime value
- Predict revenue
- Optimize resources
- Insights from purchase frequency, total spent, and recency help design targeted campaigns and product strategies.

### Issues / Observations in the Dataset:

Negative total_spent values: -5.95, -12.60, -17.90, likely due to refunds, returns, or data entry errors. These should be addressed before calculating revenue metrics.

Low purchase frequency with high total_spent: some customers have only 1 purchase but unusually high spending (e.g., customer_id 12931: 1 purchase, 1160.00 total). This may indicate bulk orders or possible anomalies.

Consistency of recency: all recency values show the same reference date (323 days), which may not reflect true differences in customer activity. You might need to verify the date used for calculations.

Outliers: there are extreme differences in purchase frequency (e.g., 104 purchases vs. 1 purchase), which may skew analyses if not normalized or segmented properly.
